In [ ]:
import uproot

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# plt.style.use("belle2")

import seaborn as sns

# import plothist
# plt.style.use("belle2")
# from plothist import make_2d_hist, plot_2d_hist_with_projections

In [ ]:
plt.rcParams.update({
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.titlesize": 16
})

# Prep-Work

### Import Data

In [ ]:
# In this notebook we only process the main signal and the generic events,
# for illustration purposes.
# You can add other backgrounds after if you wish.
samples = ["Signal","All","BB","ccbar","ddbar","ssbar","taupair","uubar","uds"]

DataFrames = {}  # define empty dictionary to hold dataframes

# Signal:
DataFrames[samples[0]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Dstree",library='pd')
# Background
for s in samples[1:]: # loop over samples
    DataFrames[s] =  uproot.concatenate("/group/belle2/users2022/amubarak/TopoAna/Completed_TopoAna/TopoAna_"+ s +".root:Dstree",library='pd')

In [ ]:
pd.set_option('display.max_rows', 200000)
pd.set_option('display.max_columns', 200000)

The line below is to look at the available variables.

In [ ]:
DataFrames["ccbar"].columns.tolist()

### Setup
The code below will be used to apply cuts to the data.  
The range of the plots.

In [ ]:
# Electron ID
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['e_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['e_electronID']>=0.95]
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_em_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_gammaveto_em_electronID']>=0.95]

# Fake D0 Suppression
#------------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_extraInfo_FastBDT']>=0.58]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_extraInfo_FastBDT']>=0.58]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_extraInfo_FastBDT']>=0.58]

# Peaking Background Removal
#----------------------------
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_diff_D0pi']>=0.15)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.15)]

# Photon Conversion
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=0.1]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=0.1]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_gammaveto_M_Correction']>=0.1]

# # Vertex Fitting
# #----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb']>=0.01]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_chiProb']>=0.01]

# Vertex Fit
#----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb_Ds_rank']==1]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_chiProb_Ds_rank']==1]

In [ ]:
print(abs(DataFrames["All"]['Ds_mcPDG']).value_counts(normalize=True,dropna=False).head(100))

# Single Variable  
The code below is to look at any specific variable closely.

### First Method

In [ ]:
Stacked = True
Density = False
Bins = 50
# i = 'Ds_diff_D0pi'
i = 'Ds_massDifference_0'
Range = [0.0, 0.25]
Op = -1
dM = -1
Hits = -1
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Comb.$'
label2= r'$NaN$'
label3= r'$D^{*0}$'
label4= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'

labels=[
        label1,
        label2,
        label3,
        label4
        ]
colors=[
        "#DD8452",
        "#C44E52",
        "#55A868",
        "#4C72B0"
        ]
data=[
      DataFrames["All"][((abs(DataFrames["All"]["Ds_mcPDG"])!=413) & (abs(DataFrames["All"]["Ds_mcPDG"])!=423) & (~DataFrames["All"]["Ds_mcPDG"].isna())) & (DataFrames["All"]['Ds_diff_D0pi']>=dM) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=0.1)][i],
      DataFrames["All"][(DataFrames["All"]["Ds_mcPDG"].isna()) & (DataFrames["All"]['Ds_diff_D0pi']>=dM) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=0.1)][i],
      DataFrames["All"][(abs(DataFrames["All"]["Ds_mcPDG"])==423) & (DataFrames["All"]['Ds_diff_D0pi']>=dM) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=0.1)][i],
      DataFrames["All"][(abs(DataFrames["All"]["Ds_mcPDG"])==413) & (DataFrames["All"]['Ds_diff_D0pi']>=dM) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=0.1)][i]
      ]

# factor = 0.5
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM)][i]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.162,color='gray',alpha=0.2)
# plt.axvline(0.162,ls='--',color='gray')

# Title
#--------
plt.title(r'$Electron \; Assigned \; Electron \; Mass$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 80000)
plt.legend(loc='upper right')
plt.show()

In [ ]:
Stacked = False
Density = True
Bins = 50
# i = 'Ds_diff_D0pi'
i = 'Ds_massDifference_0'
Range = [0.0, 0.25]
Op = -1
dM = -1
Hits = -1
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'Before $D^{*+}$ Cut'
label2= r'After $D^{*+}$ Cut'

labels=[
        label1,
        label2
        ]
colors=[
        "#007C91", "#4C6EB1"
        ]
data=[
      DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=-1) & (DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=-1)][i],
      DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.162) & (DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=-1)][i],
      ]

# factor = 0.5
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM)][i]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.162,color='gray',alpha=0.2)
# plt.axvline(0.162,ls='--',color='gray')

# Title
#--------
plt.title(r'$Electron \; Assigned \; Electron \; Mass$', loc = "left")
plt.title(r'$\bf Signal \; Events$', loc = "right")
# Label
#-------
plt.ylabel(r'$Norm. Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 80000)
plt.legend(loc='upper right')
plt.show()

In [ ]:
print(abs(DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.162) & (DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=-1)]['Ds_isSignal']).value_counts(normalize=False,dropna=False).head(100))

In [ ]:
Stacked = False
Density = False
Bins = 50
# i = 'Ds_M'
# i = 'Ds_diff_D0pi'
i = 'Ds_massDifference_0'
Range = [0.,0.25]
dM = -4
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$D^{*0} \rightarrow D^{0} [\pi^0 \rightarrow \gamma [\gamma \rightarrow e^{+} e^{-}]]$'
label2= r'$D^{*0} \rightarrow D^{0} [\gamma \rightarrow  e^{+} e^{-}]$'
label3= r'$D^{*0} \rightarrow D^{0} [\pi^0 \rightarrow \gamma  e^{+} e^{-}]$'

labels=[label1,label2,label3]
data=[DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423) & (abs(DataFrames["ccbar"]['Ds_genNStepsToDaughter_1'])==3) & (abs(DataFrames["ccbar"]['e_genMotherPDG'])==22) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)][i],
      DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423) & (abs(DataFrames["ccbar"]['Ds_genNStepsToDaughter_1'])==2) & (abs(DataFrames["ccbar"]['e_genMotherPDG'])==22) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)][i],
      DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423) & (abs(DataFrames["ccbar"]['Ds_genNStepsToDaughter_1'])==2) & (abs(DataFrames["ccbar"]['e_genMotherPDG'])==111) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)][i],
      ]

factor = 0.2
plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM)][i]), ls='--', linewidth=1.5)
plt.hist(data, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.15,ls='--',color='gray')

# Title
#--------
# plt.title(r'$BCS: Vertex \; Fit \; \chi^{2}$', loc = "left")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$m(D_s^{+})\;[GeV/c^{2}]$')
# plt.yscale("log")
plt.legend()
plt.show()

# Correlation

In [ ]:
list = ["Ds_M","D0_dM","D0_M","Ds_massDifference_0","Ds_diff_D0pi"]

In [ ]:
plt.figure(figsize=(10, 9))

heatmap = sns.heatmap(DataFrames["Signal"][list].corr(), annot=True, cmap="coolwarm",vmin=-1, vmax=1)

heatmap.set_title('Signal Correlation Heatmap', fontdict={'fontsize':18}, pad=16)

In [ ]:
plt.figure(figsize=(10, 9))

heatmap = sns.heatmap(DataFrames["ccbar"][list].corr(), annot=True, cmap="coolwarm",vmin=-1, vmax=1)

heatmap.set_title('Background Correlation Heatmap (All Background)', fontdict={'fontsize':18}, pad=16)

# 2-D Histogram

In [ ]:
# # name_x = 'Ds_massDifference_0'
# name_x = 'Ds_dM_D0pi'
# name_y = 'Ds_Angle_D0e'
# Sample = "ccbar"

# h = make_2d_hist([DataFrames[Sample][name_x], DataFrames[Sample][name_y]], bins=[50, 50], range=[[0.135,0.4],[0,3.14]])

# (
#     fig,
#     ax_2d,
#     ax_x_projection,
#     ax_y_projection,
#     ax_colorbar,
# ) = plot_2d_hist_with_projections(
#     h,
#     xlabel=r'$\Delta m\;[GeV/c^{2}]$',
#     ylabel=r'$\theta(D^{0}e^{+})\;[rad]$',
#     ylabel_x_projection="Entries",
#     xlabel_y_projection="Entries",
#     offset_x_labels=True,
#     colorbar_kwargs={"label": "Entries"},
#     figsize=(6,6)
# )